<a href="https://colab.research.google.com/github/Antique-1/deep-running-project/blob/main/dogs_vs_cats_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 이미지 분류 모델을 통한 딥러닝 실습 프로젝트

## 라이브러리 불러오기

In [1]:
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

## 데이터 로딩 및 전처리

https://drive.google.com/drive/folders/1eUjn1VMu1pPtKKXAHk7PbNuqG0H8J-7M?usp=drive_link
데이터셋 파일 링크

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
base_dir = '/content/drive/MyDrive/ST11/train_sorted'

# ImageDataGenerator를 활용한 전처리 및 증강
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

In [4]:
train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(250, 250),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(250, 250),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 20007 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


## CNN 모델 구성 및 학습

In [5]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(250,250,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 248, 248, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 124, 124, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 122, 122, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 61, 61, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 59, 59, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 29, 29, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 107648)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 107648)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    55,116,288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 55,210,049 (210.61 MB)

 Trainable params: 55,210,049 (210.61 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
 17/626 ━━━━━━━━━━━━━━━━━━━━ 4:06:31 24s/step - accuracy: 0.5115 - loss: 0.7117

KeyboardInterrupt: 

## 학습 결과 시각화

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## 예측 결과

In [ ]:
# 예측 예시: 검증 세트 중 일부 이미지 확인
sample_images, sample_labels = next(val_generator)
predictions = model.predict(sample_images)

plt.figure(figsize=(12, 12))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(sample_images[i])
    plt.title(f"실제: {'개' if sample_labels[i] else '고양이'} / 예측: {'개' if predictions[i] > 0.5 else '고양이'}")
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
test_dir = '/content/ST11/test1'  # 테스트 이미지 폴더 경로
img_size = (250, 250)

test_images = sorted(os.listdir(test_dir), key=lambda x: int(x.split('.')[0]))

test_data = []
image_ids = []

for img_name in test_images:
    img_path = os.path.join(test_dir, img_name)
    img = load_img(img_path, target_size=img_size)
    img_array = img_to_array(img) / 255.0
    test_data.append(img_array)
    image_ids.append(int(img_name.split('.')[0]))

test_data = np.array(test_data)

predictions = model.predict(test_data)
predicted_labels = (predictions > 0.5).astype(int).flatten()

submission_df = pd.DataFrame({
    'id': image_ids,
    'label': predicted_labels
})
submission_df = submission_df.sort_values('id')
submission_df.to_csv('submission.csv', index=False)
print("✅ submission.csv 저장 완료!")

## VGG16 모델을 활용한 학습

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D

# VGG16 base 모델 불러오기 (FC 층 제거, imagenet 가중치 사용)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(250,250,3))
base_model.trainable = False  # feature extractor로 사용

# 상위 모델 구성
vgg_model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

vgg_model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

vgg_model.summary()

In [ ]:
vgg_history = vgg_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)

## CNN 모델과 VGG16 모델 성능 비교

In [ ]:
plt.figure(figsize=(14, 6))

# Accuracy 비교
plt.subplot(1, 2, 1)
plt.plot(history.history['val_accuracy'], label='CNN Validation Acc')
plt.plot(vgg_history.history['val_accuracy'], label='VGG16 Validation Acc')
plt.title('Validation Accuracy Comparison')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Loss 비교
plt.subplot(1, 2, 2)
plt.plot(history.history['val_loss'], label='CNN Validation Loss')
plt.plot(vgg_history.history['val_loss'], label='VGG16 Validation Loss')
plt.title('Validation Loss Comparison')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()